<a href="https://colab.research.google.com/github/mario-rot/Introduction-to-Human-Language-Technology/blob/main/Session8_MarioRosas_LaurenTucker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Lab session 8 (Parsing) - IHLT

Students: Lauren Tucker & Mario Rosas


In [32]:
import nltk
from nltk import CFG, ChartParser

grammar = CFG.fromstring('''
  S   -> NP VB IN NP
  NP  -> NNS | JJ NNS | NP CC NP
  NNS -> "cats" | "dogs" | "mice" | NNS CC NNS 
  JJ  -> "big" | "small" | "lazy"
  CC  -> "and" | "or"
  IN  -> "with"
  VB  -> "play"
  ''')

sent = ['lazy', 'cats', 'play', 'with', 'mice']


## Bottom Up Chart Parser

In [8]:
from nltk import BottomUpChartParser

bu_parser = nltk.BottomUpChartParser(grammar,trace=1)
bu_parse = bu_parser.parse(sent)
bu_ts = []
for t in bu_parse:
    bu_ts.append(t)

|.  lazy .  cats .  play .  with .  mice .|
|[-------]       .       .       .       .| [0:1] 'lazy'
|.       [-------]       .       .       .| [1:2] 'cats'
|.       .       [-------]       .       .| [2:3] 'play'
|.       .       .       [-------]       .| [3:4] 'with'
|.       .       .       .       [-------]| [4:5] 'mice'
|>       .       .       .       .       .| [0:0] JJ -> * 'lazy'
|[-------]       .       .       .       .| [0:1] JJ -> 'lazy' *
|>       .       .       .       .       .| [0:0] NP -> * JJ NNS
|[------->       .       .       .       .| [0:1] NP -> JJ * NNS
|.       >       .       .       .       .| [1:1] NNS -> * 'cats'
|.       [-------]       .       .       .| [1:2] NNS -> 'cats' *
|.       >       .       .       .       .| [1:1] NP -> * NNS
|.       >       .       .       .       .| [1:1] NNS -> * NNS CC NNS
|[---------------]       .       .       .| [0:2] NP -> JJ NNS *
|.       [-------]       .       .       .| [1:2] NP -> NNS *
|.       [------->  

In [25]:
print("num edges = ",bu_parse.num_edges())

num edges =  44


In [9]:
print('number of trees:', len(bu_ts))

number of trees: 1


In [10]:
print(bu_ts[0])

(S (NP (JJ lazy) (NNS cats)) (VB play) (IN with) (NP (NNS mice)))


In [26]:
bu_parse.edges()

[[Edge: [0:1] 'lazy'],
 [Edge: [1:2] 'cats'],
 [Edge: [2:3] 'play'],
 [Edge: [3:4] 'with'],
 [Edge: [4:5] 'mice'],
 [Edge: [0:0] JJ -> * 'lazy'],
 [Edge: [0:1] JJ -> 'lazy' *],
 [Edge: [0:0] NP -> * JJ NNS],
 [Edge: [0:1] NP -> JJ * NNS],
 [Edge: [1:1] NNS -> * 'cats'],
 [Edge: [1:2] NNS -> 'cats' *],
 [Edge: [1:1] NP -> * NNS],
 [Edge: [1:1] NNS -> * NNS CC NNS],
 [Edge: [0:2] NP -> JJ NNS *],
 [Edge: [1:2] NP -> NNS *],
 [Edge: [1:2] NNS -> NNS * CC NNS],
 [Edge: [1:1] S  -> * NP VB IN NP],
 [Edge: [1:1] NP -> * NP CC NP],
 [Edge: [1:2] S  -> NP * VB IN NP],
 [Edge: [1:2] NP -> NP * CC NP],
 [Edge: [0:0] S  -> * NP VB IN NP],
 [Edge: [0:0] NP -> * NP CC NP],
 [Edge: [0:2] S  -> NP * VB IN NP],
 [Edge: [0:2] NP -> NP * CC NP],
 [Edge: [2:2] VB -> * 'play'],
 [Edge: [2:3] VB -> 'play' *],
 [Edge: [1:3] S  -> NP VB * IN NP],
 [Edge: [0:3] S  -> NP VB * IN NP],
 [Edge: [3:3] IN -> * 'with'],
 [Edge: [3:4] IN -> 'with' *],
 [Edge: [1:4] S  -> NP VB IN * NP],
 [Edge: [0:4] S  -> NP VB IN *

## Bottom Up Left Corner Chart Parser 

In [28]:
from nltk import BottomUpLeftCornerChartParser

bulc_parser = nltk.BottomUpLeftCornerChartParser(grammar,trace=1)
bulc_parse = bulc_parser.chart_parse(sent)
bulc_ts = []
for t in bulc_parse:
    bulc_ts.append(t)

|.  lazy .  cats .  play .  with .  mice .|
|[-------]       .       .       .       .| [0:1] 'lazy'
|.       [-------]       .       .       .| [1:2] 'cats'
|.       .       [-------]       .       .| [2:3] 'play'
|.       .       .       [-------]       .| [3:4] 'with'
|.       .       .       .       [-------]| [4:5] 'mice'
|[-------]       .       .       .       .| [0:1] JJ -> 'lazy' *
|[------->       .       .       .       .| [0:1] NP -> JJ * NNS
|.       [-------]       .       .       .| [1:2] NNS -> 'cats' *
|.       [-------]       .       .       .| [1:2] NP -> NNS *
|.       [------->       .       .       .| [1:2] NNS -> NNS * CC NNS
|[---------------]       .       .       .| [0:2] NP -> JJ NNS *
|[--------------->       .       .       .| [0:2] S  -> NP * VB IN NP
|[--------------->       .       .       .| [0:2] NP -> NP * CC NP
|.       [------->       .       .       .| [1:2] S  -> NP * VB IN NP
|.       [------->       .       .       .| [1:2] NP -> NP * CC NP
|.  

In [29]:
print("num edges = ",bulc_parse.num_edges())

num edges =  28


In [14]:
print('number of trees:', len(bulc_ts))

number of trees: 1


In [15]:
print(bulc_ts[0])

(S (NP (JJ lazy) (NNS cats)) (VB play) (IN with) (NP (NNS mice)))


In [17]:
bulc_parse.edges()

[[Edge: [0:1] 'lazy'],
 [Edge: [1:2] 'cats'],
 [Edge: [2:3] 'play'],
 [Edge: [3:4] 'with'],
 [Edge: [4:5] 'mice'],
 [Edge: [0:1] JJ -> 'lazy' *],
 [Edge: [0:1] NP -> JJ * NNS],
 [Edge: [1:2] NNS -> 'cats' *],
 [Edge: [1:2] NP -> NNS *],
 [Edge: [1:2] NNS -> NNS * CC NNS],
 [Edge: [0:2] NP -> JJ NNS *],
 [Edge: [0:2] S  -> NP * VB IN NP],
 [Edge: [0:2] NP -> NP * CC NP],
 [Edge: [1:2] S  -> NP * VB IN NP],
 [Edge: [1:2] NP -> NP * CC NP],
 [Edge: [2:3] VB -> 'play' *],
 [Edge: [0:3] S  -> NP VB * IN NP],
 [Edge: [1:3] S  -> NP VB * IN NP],
 [Edge: [3:4] IN -> 'with' *],
 [Edge: [0:4] S  -> NP VB IN * NP],
 [Edge: [1:4] S  -> NP VB IN * NP],
 [Edge: [4:5] NNS -> 'mice' *],
 [Edge: [4:5] NP -> NNS *],
 [Edge: [4:5] NNS -> NNS * CC NNS],
 [Edge: [4:5] S  -> NP * VB IN NP],
 [Edge: [4:5] NP -> NP * CC NP],
 [Edge: [0:5] S  -> NP VB IN NP *],
 [Edge: [1:5] S  -> NP VB IN NP *]]

## Left Corner Chart Parser

In [30]:
from nltk import LeftCornerChartParser

lc_parser = nltk.LeftCornerChartParser(grammar,trace=1)
lc_parse = lc_parser.chart_parse(sent)
lc_ts = []
for t in lc_parse:
    lc_ts.append(t)

|.  lazy .  cats .  play .  with .  mice .|
|[-------]       .       .       .       .| [0:1] 'lazy'
|.       [-------]       .       .       .| [1:2] 'cats'
|.       .       [-------]       .       .| [2:3] 'play'
|.       .       .       [-------]       .| [3:4] 'with'
|.       .       .       .       [-------]| [4:5] 'mice'
|[-------]       .       .       .       .| [0:1] JJ -> 'lazy' *
|[------->       .       .       .       .| [0:1] NP -> JJ * NNS
|.       [-------]       .       .       .| [1:2] NNS -> 'cats' *
|.       [-------]       .       .       .| [1:2] NP -> NNS *
|[---------------]       .       .       .| [0:2] NP -> JJ NNS *
|[--------------->       .       .       .| [0:2] S  -> NP * VB IN NP
|.       [------->       .       .       .| [1:2] S  -> NP * VB IN NP
|.       .       [-------]       .       .| [2:3] VB -> 'play' *
|[----------------------->       .       .| [0:3] S  -> NP VB * IN NP
|.       [--------------->       .       .| [1:3] S  -> NP VB * IN NP
|. 

In [31]:
print("num edges = ",lc_parse.num_edges())

num edges =  22


In [20]:
print('number of trees:', len(lc_ts))

number of trees: 1


In [21]:
print(lc_ts[0])

(S (NP (JJ lazy) (NNS cats)) (VB play) (IN with) (NP (NNS mice)))


In [23]:
lc_parse.edges()

[[Edge: [0:1] 'lazy'],
 [Edge: [1:2] 'cats'],
 [Edge: [2:3] 'play'],
 [Edge: [3:4] 'with'],
 [Edge: [4:5] 'mice'],
 [Edge: [0:1] JJ -> 'lazy' *],
 [Edge: [0:1] NP -> JJ * NNS],
 [Edge: [1:2] NNS -> 'cats' *],
 [Edge: [1:2] NP -> NNS *],
 [Edge: [0:2] NP -> JJ NNS *],
 [Edge: [0:2] S  -> NP * VB IN NP],
 [Edge: [1:2] S  -> NP * VB IN NP],
 [Edge: [2:3] VB -> 'play' *],
 [Edge: [0:3] S  -> NP VB * IN NP],
 [Edge: [1:3] S  -> NP VB * IN NP],
 [Edge: [3:4] IN -> 'with' *],
 [Edge: [0:4] S  -> NP VB IN * NP],
 [Edge: [1:4] S  -> NP VB IN * NP],
 [Edge: [4:5] NNS -> 'mice' *],
 [Edge: [4:5] NP -> NNS *],
 [Edge: [0:5] S  -> NP VB IN NP *],
 [Edge: [1:5] S  -> NP VB IN NP *]]

## Conclusions

The most efficient parser is the Left Corner Chart Parser, since it only explores 22 edges, compared to the Bottom Up Chart Parser and Bottom Up Left Corner Chart Parsers, which explore 44 and 28 edges, respectively. 

**Which edges are filtered out by each parser and why?**

When compared to the Bottom Up Chart Parser, the Bottom Up Left Corner Chart Parser filters out all self-referential edges (i.e. [0:0], [1:1], etc.). This is because ........... The Left Corner Chart Parser also filters these edges out, in addition to the following list of edges:

[0:2] NP -> NP * CC NP

[1:2] NNS -> NNS * CC NNS

[1:2] NP -> NP * CC NP

[4:5] NNS -> NNS * CC NNS

[4:5] S  -> NP * VB IN NP

[4:5] NP -> NP * CC NP

For the [1:2] rules, these omissions occur because there are already passive rules that subsume the parts of the sentence required for this edge. That is, the following rules exist:

[1:2] NNS -> 'cats' *

[1:2] NP -> NNS *

[1:2] S  -> NP * VB IN NP

Therefore, the ([1:2] NNS -> NNS * CC NNS) rule does not need to be explored because there already exists a rule that subsumes NNS for this edge, and the ([1:2] NP -> NP \* CC NP) rule does not need to be explored because there already exists a rule subsuming NP for this edge.

Similarly, the [0:2] rule in the list above does not need to be explored, since a rule subsuming NP has already been found.

In addition, none of the [4:5] rules in the list above need to be explored, since rules subsuming NNS and NP already exist for this edge.

^^^ THIS LAST ONE IS FALSE ... There is no rule subsuming NP for [4:5], so I'm now actually not sure of the pattern